## Imports

In [1]:
# Basic imports
import pandas as pd
import numpy as np

# Warnings
import warnings 
warnings.simplefilter("ignore")

In [2]:
# Plot
from IPython import display
import seaborn as sns

import matplotlib
import matplotlib.pylab as plt
from jupyterthemes import jtplot

jtplot.style('gruvboxd')
matplotlib.use('nbagg')

## Data Reading

In [3]:
from catboost.datasets import titanic

# Data Reading
df_train, df_test = titanic()
df_train.set_index('PassengerId', inplace=True)
df_test.set_index('PassengerId', inplace=True)

# Split X_train, y_train
target = 'Survived'
features = df_test.columns

y_train = df_train[target]
df_train = df_train[features]

df_train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Preprocessing

In [4]:
from robusta.preprocessing.category import *
from robusta.preprocessing.numeric import *
from robusta.preprocessing.base import *
from robusta.pipeline import *

nums = ['Age', 'Fare', 'SibSp', 'Parch']
cats = ['Pclass', 'Sex', 'Embarked']

data_prep = FeatureUnion([
        ("numeric", make_pipeline(
            ColumnSelector(nums),
            Imputer(strategy="median"),
            GaussRank(),
            ColumnRenamer(prefix='gr_'),
        )),
        ("category", make_pipeline(
            ColumnSelector(cats),
            Imputer(strategy="most_frequent"),
            LabelEncoder(),
            ColumnRenamer(prefix='le_'),
        )),
])

X_train = data_prep.fit_transform(df_train)
X_test = data_prep.transform(df_test)

X_train.head()

,gr_Age,gr_Fare,gr_SibSp,gr_Parch,le_Pclass,le_Sex,le_Embarked
PassengerId,,,,,,,
1,-0.488867,-0.968076,0.600843,-0.214091,2,1,2
2,0.584412,0.853865,0.600843,-0.214091,0,0,0
3,-0.275357,-0.454031,-0.290348,-0.214091,2,0,2
4,0.470091,0.703973,0.600843,-0.214091,0,0,2
5,0.470091,-0.379298,-0.290348,-0.214091,2,1,2


## Fold Preparation

In [5]:
from sklearn.model_selection import KFold
from robusta.resampler import *

encoder = FeatureUnion([
    ('category', make_pipeline(
        TypeSelector(['category', 'object']), 
        TargetEncoderCV(cv=4).set_params(encoder__smoothing=200.0),
    )),
    ('numeric', make_pipeline(
        TypeSelector(np.number),
        Identity(),
    )),
])

resampler = SMOTE(random_state=50, k_neighbors=30)

fold_pipe = make_pipeline(resampler, encoder)

F_train = fold_pipe.fit_transform(X_train, y_train)

F_train.sample(5, random_state=555)

,gr_Age,gr_Fare,gr_SibSp,gr_Parch,le_Pclass,le_Sex,le_Embarked
928,0.491782,1.040380,0.600843,-0.214091,0,0,2
873,0.888832,-0.271711,-0.290348,-0.214091,2,1,2
293,-0.381808,-0.274934,-0.290348,-0.214091,2,0,2
760,-0.013926,0.016949,-0.290348,-0.214091,2,1,2
126,-0.013926,-0.770943,-0.290348,-0.214091,2,1,1


## Model

In [6]:
%%time
from lightgbm import LGBMClassifier

model = LGBMClassifier()
model.fit(X_train, y_train)

CPU times: user 426 ms, sys: 8.54 ms, total: 434 ms
Wall time: 75.7 ms


/Users/pechyonkinb/anaconda/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [7]:
from sklearn.model_selection import cross_val_score

cv = 5
scoring = 'accuracy'

estimator = make_pipeline(model)
scores = cross_val_score(estimator, X_train, y_train, cv=cv, scoring=scoring)
print('{:.4f} ± {:.4f} [model]'.format(np.mean(scores), np.std(scores)))

estimator = make_pipeline(encoder, model)
scores = cross_val_score(estimator, X_train, y_train, cv=cv, scoring=scoring)
print('{:.4f} ± {:.4f} [resampler + model]'.format(np.mean(scores), np.std(scores)))

estimator = make_pipeline(resampler, model)
scores = cross_val_score(estimator, X_train, y_train, cv=cv, scoring=scoring)
print('{:.4f} ± {:.4f} [encoder + model]'.format(np.mean(scores), np.std(scores)))

# 0.8216 ± 0.0370 (model)
# 0.8261 ± 0.0336 (resampler + model)
# 0.8249 ± 0.0320 (encoder + model)

0.8317 ± 0.0303 [model]
0.8317 ± 0.0303 [resampler + model]
0.8194 ± 0.0261 [encoder + model]


In [8]:
estimator = make_pipeline(model)

## Submit (Out-of-Fold & Test Predictions)

In [9]:
%%time
from robusta.crossval import cross_val_stack

y_oof, y_sub = cross_val_stack(estimator, cv, X_train, y_train, None, X_test)

               gr_Age   gr_Fare  gr_SibSp  gr_Parch  le_Pclass  le_Sex  \
PassengerId                                                              
1           -0.488867 -0.968076  0.600843 -0.214091          2       1   
2            0.584412  0.853865  0.600843 -0.214091          0       0   
3           -0.275357 -0.454031 -0.290348 -0.214091          2       0   
4            0.470091  0.703973  0.600843 -0.214091          0       0   
5            0.470091 -0.379298 -0.290348 -0.214091          2       1   
6           -0.013926 -0.314077 -0.290348 -0.214091          2       1   
7            1.155838  0.661086 -0.290348 -0.214091          0       1   
8           -1.425723  0.175234  1.246354  0.668357          2       1   
9           -0.237116 -0.159855 -0.290348  1.092514          2       0   
10          -0.976632  0.454031  0.600843 -0.214091          1       0   
11          -1.239213  0.100021  0.600843  0.668357          2       0   
12           1.293579  0.351950 -0.290

In [23]:
import os

path = os.path.join(os.getcwd(), 'pred')
if not os.path.exists(path):
    os.mkdir(path)

sub_path = os.path.join(path, '0 sub baseline.csv')
oof_path = os.path.join(path, '0 oof baseline.csv')

y_sub.to_csv(sub_path, header=True)
y_oof.to_csv(oof_path, header=True)

### Score:
### `[CV] 0.8249 ± 0.0314`
### `[LB] 0.7656`